# Logistic Regression
* Being a "fit" data scientist should be fine if we are working with linear models
* Life with be so much better if you used an experiment tracker like MLFlow, WandB, Comet, etc.
* Lol, I used a multiclass classification dataset for a Log Reg model... I was just testing how everything works anyway.
---

### Setup

In [28]:
import torch as t
import torch.nn.functional as F 
from torch.autograd import grad
from torch.utils.data import Dataset, DataLoader

from sklearn import linear_model
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import numpy as np

from dataclasses import dataclass

from rich import print

In [2]:
# For consistent results
t.manual_seed(123)

In [3]:
device = t.device("cuda" if t.cuda.is_available() else "cpu")

In [4]:
device

device(type='cuda')

### Config

In [9]:
@dataclass
class Config:
    input_dim: int = 10
    output_dim: int = 1
    device: t.device = device
    num_epochs: int = 20
    batch_size: int = 10
    lr = 0.05


cfg = Config()

In [6]:
cfg.device

device(type='cuda')

### Loading Data
- [ ] Do everything similarly using sklearn

In [25]:
diabetes = load_diabetes(scaled=False)

# Aside from Random Split, I can try: KFold, StratifiedKFold, StratifiedShuffleSplit, etc.
X_train, X_test, y_train, y_test = train_test_split(diabetes.data,
                                                    diabetes.target,
                                                    test_size=0.2,
                                                    random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                y_train,
                                                test_size=0.2,
                                                random_state=42) 

In [23]:
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_train.dtype)
print(y_train.dtype)

In [10]:
class DiabetesDataset(Dataset):
    '''
        Diabetes Dataset for PyTorch models
    '''
    def __init__(self, 
                X: np.ndarray,
                y: np.ndarray,
                cfg: Config):
        
        self.cfg = cfg

        self.features = t.tensor(X, device=self.cfg.device, dtype=t.float64)
        self.labels = t.tensor(y, device=self.cfg.device, dtype=t.float64)
    
    def __getitem__(self, index):
        x = self.features[index]
        y = self.labels[index]

        return x, y
    
    def __len__(self):
        return self.labels.shape[0]



In [11]:
diabetes_dataset = DiabetesDataset(X_train, y_train, cfg)

train_loader = DataLoader(
    dataset=diabetes_dataset,
    batch_size=cfg.batch_size,
    shuffle=True
)


### Logistic Regression using PyTorch

##### Trying out matmul with grad

In [25]:
x = t.randn(10, requires_grad=True, device=device)
weights = t.rand(10, requires_grad=True, device=device)
# small positive initialization
bias = t.ones(10, device=device) * 0.1

print(f"Tensor pertaining to input: \n {x}")
print(f"Tensor pertaining to weights: \n {weights}")
print(f"Tensor pertaining to bias: \n {bias}")



Tensor pertaining to input: 
 tensor([ 1.3391,  0.2052, -1.6879,  0.5103, -0.3458,  0.6455,  1.5735,  0.3519,
         1.1298,  0.0098], device='cuda:0', requires_grad=True)

Tensor pertaining to weights: 
 tensor([0.6396, 0.1476, 0.8214, 0.4767, 0.6468, 0.5723, 0.1221, 0.2315, 0.6346,
        0.9634], device='cuda:0', requires_grad=True)

Tensor pertaining to bias: 
 tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.1000], device='cuda:0')

In [26]:
z = x @ weights + bias
activated_z = t.sigmoid(z)

print(z)
print(activated_z)

tensor([0.9896, 0.9896, 0.9896, 0.9896, 0.9896, 0.9896, 0.9896, 0.9896, 0.9896,
        0.9896], device='cuda:0', grad_fn=<AddBackward0>)

tensor([0.7290, 0.7290, 0.7290, 0.7290, 0.7290, 0.7290, 0.7290, 0.7290, 0.7290,
        0.7290], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [ ]:
x = t.randn((cfg.batch_size, cfg.input_dim), device=cfg.device)
print(x.dtype)


with t.inference_mode():
    output = model(x)
    print(output)
    print(output.device)

##### Model

In [12]:

class LogisticRegression(t.nn.Module):
    '''
        Simple LogisticRegression using PyTorch 
    '''
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.linear = t.nn.Linear(self.cfg.input_dim, 
                                 self.cfg.output_dim,
                                 dtype=t.float64)

    def forward(self, x):
        logits = self.linear(x)
        probs = t.sigmoid(logits)
        
        return probs



##### Training Loop

In [ ]:

model = LogisticRegression(cfg).to(device)
optimizer = t.optim.SGD(model.parameters(), lr=cfg.lr)

for epoch in range(cfg.num_epochs):

    # prepare model for training
    model = model.train()   
    for batch_idx, (features, labels) in enumerate(train_loader):
        
        probs = model(features)

        loss = F.binary_cross_entropy(probs, labels.view(probs.shape))
        
        # So the gradients do not accumulate...
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Log
        print(f"Epoch: {epoch}/{cfg.num_epochs} | Batch: {batch_idx}/{len(train_loader)} | Train loss: {loss}")

### Logistic Regression using Sklearn

In [29]:
clf = linear_model.LogisticRegression()

clf.fit(X_train, y_train)

# Predict on the testing set
y_pred = clf.predict(X_test)

# Evaluate the model using various evaluation metrics
print("Accuracy score:", accuracy_score(y_test, y_pred))
print("Classification report:\n", classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

/home/therealmolf/.virtualenvs/model-board/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy score: 0.0

/home/therealmolf/.virtualenvs/model-board/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/therealmolf/.virtualenvs/model-board/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/therealmolf/.virtualenvs/model-board/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg

Classification report:
               precision    recall  f1-score   support

        37.0       0.00      0.00      0.00       1.0
        40.0       0.00      0.00      0.00       0.0
        42.0       0.00      0.00      0.00       1.0
        45.0       0.00      0.00      0.00       0.0
        48.0       0.00      0.00      0.00       2.0
        49.0       0.00      0.00      0.00       0.0
        52.0       0.00      0.00      0.00       2.0
        53.0       0.00      0.00      0.00       0.0
        55.0       0.00      0.00      0.00       0.0
        57.0       0.00      0.00      0.00       0.0
        59.0       0.00      0.00      0.00       0.0
        60.0       0.00      0.00      0.00       1.0
        61.0       0.00      0.00      0.00       1.0
        63.0       0.00      0.00      0.00       2.0
        64.0       0.00      0.00      0.00       1.0
        65.0       0.00      0.00      0.00       0.0
        66.0       0.00      0.00      0.00       0.0
        67.0       0.00      0.00      0.00       1.0
        68.0       0.00      0.00      0.00       1.0
        69.0       0.00      0.00      0.00       1.0
        70.0       0.00      0.00      0.00       1.0
        71.0       0.00      0.00      0.00       0.0
        72.0       0.00      0.00      0.00       3.0
        77.0       0.00      0.00      0.00       1.0
        84.0       0.00      0.00      0.00       2.0
        85.0       0.00      0.00      0.00       0.0
        87.0       0.00      0.00      0.00       1.0
        88.0       0.00      0.00      0.00       0.0
        89.0       0.00      0.00      0.00       1.0
        90.0       0.00      0.00      0.00       3.0
        91.0       0.00      0.00      0.00       1.0
        94.0       0.00      0.00      0.00       2.0
        95.0       0.00      0.00      0.00       1.0
        96.0       0.00      0.00      0.00       2.0
        98.0       0.00      0.00      0.00       1.0
        99.0       0.00      0.00      0.00       1.0
       101.0       0.00      0.00      0.00       1.0
       102.0       0.00      0.00      0.00       1.0
       104.0       0.00      0.00      0.00       0.0
       107.0       0.00      0.00      0.00       1.0
       108.0       0.00      0.00      0.00       1.0
       109.0       0.00      0.00      0.00       0.0
       110.0       0.00      0.00      0.00       1.0
       111.0       0.00      0.00      0.00       1.0
       113.0       0.00      0.00      0.00       1.0
       115.0       0.00      0.00      0.00       0.0
       118.0       0.00      0.00      0.00       1.0
       122.0       0.00      0.00      0.00       1.0
       124.0       0.00      0.00      0.00       0.0
       127.0       0.00      0.00      0.00       0.0
       128.0       0.00      0.00      0.00       1.0
       129.0       0.00      0.00      0.00       1.0
       131.0       0.00      0.00      0.00       0.0
       135.0       0.00      0.00      0.00       1.0
       136.0       0.00      0.00      0.00       1.0
       137.0       0.00      0.00      0.00       0.0
       139.0       0.00      0.00      0.00       0.0
       140.0       0.00      0.00      0.00       2.0
       143.0       0.00      0.00      0.00       0.0
       144.0       0.00      0.00      0.00       0.0
       151.0       0.00      0.00      0.00       1.0
       153.0       0.00      0.00      0.00       1.0
       156.0       0.00      0.00      0.00       1.0
       158.0       0.00      0.00      0.00       1.0
       164.0       0.00      0.00      0.00       1.0
       168.0       0.00      0.00      0.00       2.0
       170.0       0.00      0.00      0.00       1.0
       171.0       0.00      0.00      0.00       1.0
       172.0       0.00      0.00      0.00       1.0
       173.0       0.00      0.00      0.00       0.0
       178.0       0.00      0.00      0.00       0.0
       180.0       0.00      0.00      0.00       1.0
       181.0       0.00      0.00

Confusion matrix:
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]